In [ ]:
# Andy fordjour, Ari Kaminski, Bernard Ong
# This code takes Reviews from a csv file and add or delete from the file using the various function

import csv
import sys
from textblob import TextBlob
import pandas as pd
import requests
from bs4 import BeautifulSoup

FILENAME = "Reviews.csv"

def display_menu():
    """
    Displays_menu function displays the menu of the various functions to be executed
    """
    print("COMMAND MENU")
    print("list - Display all Reviews")
    print("view - View a review")
    print("total - View all entries sentiment score.")
    print("newegg - Analyze a product on newegg")
    print("exit - Exit program")



def list():
    """
    The list function displays the list of Reviews by reviwer username
    """
    
    with open(FILENAME, newline="") as file:
        reader = csv.DictReader(file)
        i = 1
        print()
        for review in reader:
            print(str(i) + ". " + review['name'])
            i += 1
        print()


        
def view():
    """
    The view list ask for the Review number and displays all informtion about the Review
    """
    name = input("name: ")
   
    with open(FILENAME, newline="") as file:
        reader = csv.DictReader(file)       
        for row in reader:
            if row['name'] == name:
                print(row['review'])
            
def totalTextBlob():
    
    """The total text blob function reads the entire csv file and assigns a score for each review"""
    
    with open(FILENAME, 'r') as csvfile:
        rows = csv.DictReader(csvfile)
        for row in rows:        
            sentence = row['review']
            blob = TextBlob(sentence)
            print(row['name'], "'s review Sentiment :",float("{:.5f}".format(blob.sentiment.polarity)))
            
def newegg_rater():
    '''
    This function takes a newegg item and rates it
    '''
    
    # URL = "https://www.newegg.com/amd-ryzen-5-5600x/p/N82E16819113666?Description=amd&cm_re=amd-_-19-113-666-_-Product&quicklink=true"
    URL = input("enter the link for a newegg item!")
    
    r = requests.get(URL)

    soup = BeautifulSoup(r.content, 'html5lib')

    entries=[]  # a list to store entries

    table = soup.find('div', attrs = {'class':'comments'})

    
    #print( table)

    for col in table:
        entry = {}
        for row in col.findAll('div', attrs = {'class':'comments-name'}): #gets usernames
            entry['name'] = row.text
        for row in col.findAll('div', attrs = {'class':'comments-content'}): #gets review text
            entry['review'] = row.text
        entries.append(entry)

      
    with open(FILENAME, 'w', newline='') as f:
        w = csv.DictWriter(f,['name','review'])
        w.writeheader()
        for entry in entries:
            w.writerow(entry)
    return entries


def amazon_rater():
    '''
    This function takes a newegg item and rates it
    '''
    
    # URL = "https://www.newegg.com/amd-ryzen-5-5600x/p/N82E16819113666?Description=amd&cm_re=amd-_-19-113-666-_-Product&quicklink=true"
    URL = input("enter the link for an amazon item!")
    
    user_agent = { 
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36', 
    'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 
    'Accept-Language' : 'en-US,en;q=0.5',
    'Accept-Encoding' : 'gzip', 
    'DNT' : '1', # Do Not Track Request Header 
    'Connection' : 'close'
        }
    
    r = requests.get(URL, headers=user_agent )

    soup = BeautifulSoup(r.content, 'html.parser')

    entries=[]  # a list to store entries

    table = soup.find('div', attrs = {'id':'cm_cr-review_list'})

    #print( table.prettify())
    
    #print(table)

    for col in table:
        entry = {}
        for row in col.findAll('div', attrs = {'class':'a-profile-content'}): #gets usernames
            entry['name'] = row.text
        for row in col.findAll('div', attrs = {'class':'a-row a-spacing-small review-data'}): #gets review text
            entry['review'] = row.text
        entries.append(entry)

      
    with open(FILENAME, 'w', newline='') as f:
        w = csv.DictWriter(f,['name','review'])
        w.writeheader()
        for entry in entries:
            w.writerow(entry)
    

              
def main():
    """
    The main function ask user to select from menu and displays the command. 
    """
    print("Welcome to the Review Sentiment Program!\n")
    display_menu()
    print()
    
    command = input("Enter Command: ")
    
  
    while (command.lower() != "exit"):        
        if command.lower() == "list":
            list()
        elif command.lower() == "total":
            totalTextBlob()            
            
            print()
        elif command.lower() =="view":
            view()
            print()
        elif command.lower() =="newegg":            
            newegg_rater()
            print()
        elif command.lower() =="amazon":            
            amazon_rater()
            print()
        else:
            print("Not a valid command.  Please try again.\n")            
        command = input("Enter Command: ")
               
    print("Bye!")     
        
if __name__ == "__main__":
    main()
